<a href="https://colab.research.google.com/github/Sornambal/PGAGI-ASSIGNMENT---TalentScout-AI-Hiring-Assistant-Chatbot/blob/main/AIML_INTERN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install --quiet gradio openai cryptography


In [5]:
import os, getpass

# OPTION 1: Hide input (recommended)
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key (or leave blank for fallback): ")

# OPTION 2 (alternative): Paste directly (not recommended if sharing notebook)
# os.environ["OPENAI_API_KEY"] = "sk-your-api-key"


Enter your OpenAI API key (or leave blank for fallback): ··········


In [6]:
import re, os, json, sqlite3
from datetime import datetime
from cryptography.fernet import Fernet
import openai

# Get API Key
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_KEY:
    openai.api_key = OPENAI_KEY

# Encryption key setup
FERNET_FILE = "fernet.key"
if not os.path.exists(FERNET_FILE):
    key = Fernet.generate_key()
    with open(FERNET_FILE, "wb") as f:
        f.write(key)
else:
    with open(FERNET_FILE, "rb") as f:
        key = f.read()
fernet = Fernet(key)


In [7]:
def init_db():
    conn = sqlite3.connect("talentscout.db", check_same_thread=False)
    c = conn.cursor()
    c.execute("""
    CREATE TABLE IF NOT EXISTS candidates (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        created_at TEXT,
        name BLOB,
        email BLOB,
        phone BLOB,
        experience TEXT,
        position TEXT,
        location TEXT,
        tech_stack TEXT,
        questions TEXT
    )
    """)
    conn.commit()
    return conn

conn = init_db()

def encrypt(value: str):
    return fernet.encrypt(value.encode()) if value else None

def decrypt(value: bytes):
    return fernet.decrypt(value).decode() if value else ""


In [8]:
def validate_email(email): return re.match(r"^[^@]+@[^@]+\.[^@]+$", email)
def validate_phone(phone): return re.match(r"^\+?\d[\d\-\s]{6,}$", phone)

def save_candidate(data):
    c = conn.cursor()
    c.execute("""
    INSERT INTO candidates (created_at, name, email, phone, experience, position, location, tech_stack, questions)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        datetime.utcnow().isoformat(),
        encrypt(data.get("name")),
        encrypt(data.get("email")),
        encrypt(data.get("phone")),
        data.get("experience"),
        data.get("position"),
        data.get("location"),
        json.dumps(data.get("tech_stack", [])),
        json.dumps(data.get("questions", {}))
    ))
    conn.commit()
    return c.lastrowid

def parse_stack(stack_str):
    return [s.strip() for s in re.split(r"[,;\n]+", stack_str) if s.strip()]

def get_openai_questions(tech, years, position, count=4):
    prompt = f"""
    Generate {count} concise technical interview questions for {tech}.
    Candidate: {years} years of experience, applying for {position}.
    Return as a numbered list.
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini" if OPENAI_KEY else "gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=400,
        )
        return response.choices[0].message.content.strip().split("\n")
    except Exception as e:
        print("⚠️ OpenAI fallback:", e)
        return fallback_questions(tech)

def fallback_questions(tech):
    base = [
        f"What are key principles of {tech}?",
        f"Explain common mistakes in {tech}.",
        f"How would you optimize performance in {tech}?",
        f"Describe a real-world use case of {tech}."
    ]
    if "python" in tech.lower():
        base += ["What is GIL?", "Difference between list and tuple in Python?"]
    if "django" in tech.lower():
        base += ["Explain Django ORM.", "What are middlewares in Django?"]
    if "react" in tech.lower():
        base += ["What are hooks in React?", "How to optimize rendering?"]
    return base[:4]

def generate_questions(stack, years, position):
    return {tech: get_openai_questions(tech, years, position) for tech in stack}


In [9]:
import gradio as gr

session = {"data": {}, "questions": {}}
END_WORDS = {"end", "exit", "bye", "stop", "quit"}

def reset():
    session["data"] = {
        "name": None, "email": None, "phone": None,
        "experience": None, "position": None,
        "location": None, "tech_stack": None
    }
    session["questions"] = {}
    return "👋 Hi, I’m TalentScout Hiring Assistant! Let’s begin.\nPlease tell me your *full name*."

def handle(user_input):
    text = user_input.strip()

    # Exit handling
    if text.lower() in END_WORDS:
        if session["data"].get("name"):
            save_candidate({**session["data"], "questions": session["questions"]})
            return "✅ Thank you! Your responses are saved. We’ll contact you soon."
        return "Goodbye!"

    # --- 🧠 Smart Name Detection ---
    if not session["data"].get("name"):
        lower = text.lower()
        # If user says "I am ..." or "My name is ..."
        if "my name is" in lower or "i am" in lower:
            # Extract words after "is" or "am"
            parts = lower.split("is")[-1] if "is" in lower else lower.split("am")[-1]
            name = parts.strip().title()
            session["data"]["name"] = name
            return f"Nice to meet you, {name}! Please provide your *email address*."
        # If they just typed a short name (<=3 words)
        elif len(text.split()) <= 3:
            session["data"]["name"] = text.title()
            return f"Thanks, {session['data']['name']}! Please provide your *email address*."

    # --- 🧩 Handle colon-style structured inputs (name:, email:, etc.) ---
    if ":" in text:
        key, val = [x.strip() for x in text.split(":", 1)]
        key_lower = key.lower()

        if "name" in key_lower:
            session["data"]["name"] = val.title()
            return f"Got it, {session['data']['name']}. Please share your *email address*."
        elif "email" in key_lower:
            if validate_email(val):
                session["data"]["email"] = val
                return "Email saved. Please provide your *phone number* (include country code)."
            else:
                return "❌ Invalid email. Try again."
        elif "phone" in key_lower:
            if validate_phone(val):
                session["data"]["phone"] = val
                return "Great! How many *years of experience* do you have?"
            else:
                return "❌ Invalid phone number."
        elif "experience" in key_lower:
            session["data"]["experience"] = val
            return "Nice! What *position* are you applying for?"
        elif "position" in key_lower:
            session["data"]["position"] = val
            return "Understood. Where are you *currently located*?"
        elif "location" in key_lower:
            session["data"]["location"] = val
            return "Thanks! Finally, what’s your *tech stack*? (e.g., Python, Django, React)"
        elif "tech" in key_lower:
            stack = parse_stack(val)
            session["data"]["tech_stack"] = stack
            q = generate_questions(
                stack,
                session["data"].get("experience", "0"),
                session["data"].get("position", "Candidate")
            )
            session["questions"] = q
            return f"✅ Tech stack saved: {', '.join(stack)}.\nI’ve generated questions. Type 'show' to view them."

    # --- 🧾 Show questions ---
    if text.lower().startswith("show"):
        if not session["questions"]:
            return "Please provide your tech stack first."
        reply = ""
        for tech, qs in session["questions"].items():
            reply += f"\n**{tech}**:\n" + "\n".join([f"{i+1}. {q}" for i, q in enumerate(qs)]) + "\n"
        return reply

    # --- 🧩 Missing info fallback prompts ---
    missing = [k for k, v in session["data"].items() if not v]
    if missing:
        prompts = {
            "name": "Please tell me your *full name*.",
            "email": "Please provide your *email address*.",
            "phone": "Please provide your *phone number* (include country code).",
            "experience": "Please specify your *years of experience*.",
            "position": "Which *position* are you applying for?",
            "location": "Where are you *currently located*?",
            "tech_stack": "What’s your *tech stack*? (e.g., Python, Django, React)"
        }
        return prompts[missing[0]]

    # --- 🏁 Final step ---
    return "Type 'show' to see questions or 'end' to finish the conversation."


In [10]:
import re

def validate_email(email: str) -> bool:
    """Check if the email format is valid"""
    return bool(re.match(r"^[^@]+@[^@]+\.[^@]+$", email.strip()))

def validate_phone(phone: str) -> bool:
    """Check if the phone number looks valid (digits and optional +country code)"""
    return bool(re.match(r"^\+?\d[\d\-\s]{6,}$", phone.strip()))

def parse_stack(stack_str: str):
    """Split comma or semicolon separated tech stack"""
    return [s.strip() for s in re.split(r"[,;\n]+", stack_str) if s.strip()]


In [11]:
import json, re
import openai
import os

# ✅ Make sure OpenAI key is set (optional)
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_KEY:
    openai.api_key = OPENAI_KEY

# 🧠 Fallback question generator for offline or keyless mode
def fallback_questions(tech):
    """Return some basic predefined questions for known tech stacks"""
    tech_lower = tech.lower()
    base = [
        f"What are key principles of {tech}?",
        f"Explain common mistakes in {tech}.",
        f"How would you optimize performance in {tech}?",
        f"Describe a real-world use case of {tech}."
    ]
    if "python" in tech_lower:
        base += ["Explain Python’s GIL.", "Difference between list and tuple."]
    if "django" in tech_lower:
        base += ["What is Django ORM?", "Explain Django middleware."]
    if "react" in tech_lower:
        base += ["What are React hooks?", "How does the virtual DOM work?"]
    if "node" in tech_lower:
        base += ["Explain the event loop in Node.js.", "How do you handle async errors?"]
    return base[:4]

# ✅ OpenAI question generator (if API key available)
def get_openai_questions(tech, years, position, count=4):
    prompt = f"""
    Generate {count} concise technical interview questions for {tech}.
    Candidate has {years} years of experience applying for {position}.
    Return a simple numbered list (no answers).
    """
    if not OPENAI_KEY:
        return fallback_questions(tech)
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=400,
        )
        output = response.choices[0].message.content.strip()
        lines = [l.strip("-• ") for l in output.splitlines() if l.strip()]
        return lines[:count] or fallback_questions(tech)
    except Exception as e:
        print("⚠️ OpenAI error:", e)
        return fallback_questions(tech)

# ✅ Main function that your chatbot calls
def generate_questions(stack, years, position):
    """Generate a dictionary of {tech: [questions]}"""
    results = {}
    for tech in stack:
        results[tech] = get_openai_questions(tech, years, position)
    return results


In [ ]:
with gr.Blocks() as app:
    gr.Markdown("# 🤖 TalentScout Hiring Assistant Chatbot")
    chat = gr.Chatbot()
    msg = gr.Textbox(placeholder="Type here... (e.g., 'email: you@example.com')", lines=2)
    btn = gr.Button("Send")
    clear = gr.Button("Reset Chat")

    state = gr.State([])

    def chat_flow(message, history):
        reply = handle(message)
        history.append((message, reply))
        return "", history

    def clear_chat():
        reset()
        return [], "Session reset."

    btn.click(chat_flow, [msg, chat], [msg, chat])
    clear.click(lambda: reset(), None, chat)

reset()
app.launch(debug=True, share=True)


/tmp/ipython-input-1948965398.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7fa3bb50d34763b1ab.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


⚠️ OpenAI error: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

⚠️ OpenAI error: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

